In [7]:
import pandas as pd

# data입력
data =     [list(map(int,"0 0 5 0 0 8 0 2 0".split(" ")))]
data.append(list(map(int,"0 0 9 0 0 6 0 5 0".split(" "))))
data.append(list(map(int,"0 3 6 0 0 0 1 0 0".split(" "))))
data.append(list(map(int,"9 0 0 0 0 0 0 7 0".split(" "))))
data.append(list(map(int,"1 0 0 0 5 0 0 0 9".split(" "))))
data.append(list(map(int,"0 8 0 0 0 2 0 0 5".split(" "))))
data.append(list(map(int,"0 0 8 0 0 0 5 3 0".split(" "))))
data.append(list(map(int,"0 9 0 8 0 0 4 0 0".split(" "))))
data.append(list(map(int,"0 2 0 4 0 0 6 0 0".split(" "))))

df = pd.DataFrame(data)
df


,0,1,2,3,4,5,6,7,8
0,0,0,5,0,0,8,0,2,0
1,0,0,9,0,0,6,0,5,0
2,0,3,6,0,0,0,1,0,0
3,9,0,0,0,0,0,0,7,0
4,1,0,0,0,5,0,0,0,9
5,0,8,0,0,0,2,0,0,5
6,0,0,8,0,0,0,5,3,0
7,0,9,0,8,0,0,4,0,0
8,0,2,0,4,0,0,6,0,0


In [8]:
num = [1,2,3,4,5,6,7,8,9]

def search_filter(list_1):  # 후보숫자 찾기
    return list(filter(lambda x: x not in list_1, num))

def select_circle(x,y):  # 3*3셀의 인덱스 찾기
    point_x = (x // 3) * 3  
    point_y = (y // 3) * 3 
    list_circle = []
    for i in range(point_x, point_x + 3):
        for j in range(point_y, point_y + 3):
            list_circle.append([i,j])
    return list_circle

def make_list():    # 비어있는 모든 셀에 대한 후보숫자 만들기
    loc_zero = []
    for x in range(9):
        for y in range(9):
            if df[x][y] == 0:
                loc_zero.append([y,x])  # 비어있는 셀의 위치정보 
    
    list_candidate = []
    for inx in loc_zero:   # 비어있는 셀에 후보숫자 생성하기
        list_h = search_filter(list(df.iloc[inx[0],:]))
        list_v = search_filter(list(df.iloc[:,inx[1]]))
        point_x = (inx[0] // 3) * 3  
        point_y = (inx[1] // 3) * 3 
        list_circle = []
        for i in range(point_x, point_x + 3):
            for j in range(point_y, point_y + 3):
                list_circle.append(df.iloc[i,j])
        list_c = search_filter(list_circle)
        intersection = list(set(list_h) & set(list_v) & set(list_c))
        if len(intersection) > 0:  # 가로,세로,3*3셀에 유효한 숫자 찾기
            list_candidate.append([inx[0], inx[1], intersection])
    return list_candidate

def delete_list(list_candi, list_deleted):  # 숫자가 결정된 셀에 대한 관련셀 정리하기
    for i in list_deleted:
        x = i[0] # 좌표x
        y = i[1] # 좌표y
        z = i[2] # 후보에서 제외된 숫자 1개(셀에 1개 숫자 결정된 것임)
        list_circle = select_circle(x,y) # 3*3셀의 인덱스 생성
        for index, j in enumerate(list_candi): # 후보숫자 리스트 검색
            list_del = j[2]  # 후보숫자 리스트
            if (x == j[0] or y == j[1]) and z in list_del: # 결정된 셀과 관련된 가로, 세로셀의 후보숫자 정리
                list_del.remove(z)
                list_candi[index][2] = list_del
            for k in list_circle:  # 결정된 셀과 관련있는 3*3셀에서 후보숫자 정리
                if j[0] == k[0] and j[1] == k[1] and z in list_del:
                    list_del.remove(z)
                    list_candi[index][2] = list_del
    return list_candi

def second_saved(list_candi):   # 가로,세로,3*3 셀에서 가능한 숫자 찾기 
    df1 = pd.DataFrame(list_candi, columns=['a','b','c'])
    for index, val in enumerate(list_candi): # 가로,세로,3*3셀에서 가능한 숫자 1개 찾기
        point_x = val[0]
        point_y = val[1]
        number_list = val[2]
        cnt1 = df1.c[df1['a'] == point_x].sum()  # 가로
        cnt2 = df1.c[df1['b'] == point_y].sum()  # 세로
        x = (point_x // 3) * 3  
        y = (point_y // 3) * 3 
        cnt3 = df1.c[(df1['a'] >= x) & (df1['a'] <= x+3) & (df1['b'] >= y) & (df1['b'] <= y+3)].sum() # 3*3
        for i in number_list:
            if cnt1.count(i) == 1 or cnt2.count(i) == 1 or cnt3.count(i) == 1:
                list_candi[index][2] = [i]
                break
    return list_candi

def save_number(list_candi): # 결정된 셀에 대한 숫자 저장하기
    while True:
        list_deleted = []
        for index, val in enumerate(list_candi):
            x = val[0]
            y = val[1]
            z = val[2]   # 후보숫자 리스트
            if len(z) == 1:
                df.iloc[x,y] = z[0]
                del list_candi[index]
                list_deleted.append([x,y,z[0]])
        if len(list_deleted) == 0:
            break
        list_candi = delete_list(list_candi, list_deleted)
    return list_candi
        
# start -->

list_candi = make_list() # 빈 셀에 대한 후보숫자 만들기
while True:
    if len(list_candi) == 0:
        break
    list_candi = save_number(list_candi)
    list_candi = second_saved(list_candi)
            
df    

,0,1,2,3,4,5,6,7,8
0,4,7,5,3,1,8,9,2,6
1,8,1,9,2,4,6,7,5,3
2,2,3,6,5,7,9,1,8,4
3,9,5,3,6,8,4,2,7,1
4,1,6,2,7,5,3,8,4,9
5,7,8,4,1,9,2,3,6,5
6,6,4,8,9,2,1,5,3,7
7,3,9,7,8,6,5,4,1,2
8,5,2,1,4,3,7,6,9,8
